In [ ]:
import os
os.chdir("../")
os.getcwd()

In [ ]:
from attacker.query import *
from attacker.utils import *
from victim.__init__ import *

set_seed(42)

# Query victim model & Train attacker model

In [ ]:
config = {
    "batch_size": 500,
    "learning_rate": 0.008,
    "epochs": 80,
    "query_size":10000,
    "query_type": 'random',
    "victim": { "data": CIFAR_10,  "model_name": RESNET50 },
    "attacker": RESNET34,
    "klogits": 3
}

In [ ]:
# get dataset in dataloader
trainloader, testloader, outputs = get_dataloader(config["victim"]["data"])

# query & save test data
querytestloader = query_victim(config["victim"], outputs, testloader, len(testloader.dataset), train=False)

# query train data
querytrainloader = query_victim(config["victim"], outputs, trainloader, config["query_size"], config["klogits"], q_type=config["query_type"])

# initialize attacker model
attacker = get_model(config["attacker"], outputs)

# train attacker model
attacker_result = training(attacker, querytrainloader, querytestloader, config["epochs"],
                           optimizer=torch.optim.Adam(attacker.parameters(), lr=config["learning_rate"]),
                           loss=torch.nn.CrossEntropyLoss())

# save & visualize model inference
title = f'Trial-A_{config["attacker"]}_{config["victim"]["model_name"]}_{config["victim"]["data"]}{config["query_type"]}_{config["query_size"]}_k{config["klogits"]}'
percent = SaveVisualize(attacker, attacker_result,title )
results_dict={"Victim":config["victim"]["model_name"],
                "Dataset":config["victim"]["data"],
                "QueryType": config["query_type"],
                "QuerySize": config["query_size"],
                "Attacker": 'resnet34',
                "Train Loss": percent[0],
                "Train Accuracy": percent[1],
                "Test Loss": percent[2],
                "Test Accuracy": percent[3]}
print(results_dict)

# Parameters

In [ ]:
parameters = {
        "query_size": [10000, 20000, 30000, 40000, 50000],
        "query_type": ['random''coreset'],
        "victim":[{ "data": CIFAR_10, "model_name": RESNET50},
                  {"data": CIFAR_100, "model_name": RESNET50 }],
        "attacker":[RESNET34],
        "klogits": [0]
}

Case 2 : Victim vs Attacker
    Dataset - Cifar10
    klogits - 0
    QueryType - Random
    Query Size - 10k, 30k, 50k
    Victim - R34, R50, VGG19
    Attacker - R34, R50, VGG19

Case 3 : K-Logits vs Labels
    Dataset - Cifar10
    klogits - 0, 3, 10
    QueryType - Random
    Query Size - 10k, 30k, 50k
    Victim - R34
    Attacker - R50!

# Investigations

In [ ]:
Results = []
# Iterate through Victim Model & Dataset
for victim in parameters["victim"]:
    print('---------------------------------------------------------------------------')
    # get dataset in dataloader
    trainloader, testloader, outputs = getDataset(victim["data"])

    # Iterate through logit size
    for k in parameters["klogits"]:
        # query test data
        querytestloader = QueryVictim(victim, outputs, testloader, len(testloader.dataset), train=False)

        # Iterate through Attacker Model
        for attacker_type in parameters["attacker"]:
            # Iterate Through Query Type
            for querytype in parameters["query_type"]:
                # Iterate Through Query Size
                for size in parameters["query_size"]:
                    print('-----------------------------------------------------------------------------')
                    print(f'-----------------------Dataset: {victim["data"]}----------------------------')
                    print(f'--------Victim: {victim["model_name"]} Attacker: {attacker_type}------------')
                    print(f'---------------Query Type: {querytype} Query Size: {size}-------------------')

                    # query train data
                    querytrainloader = query_victim(victim, outputs, trainloader, size, k, q_type=querytype)

                    # initialize attacker model
                    attacker = get_model(attacker_type, outputs)

                    # train attacker model
                    attacker_result = training(attacker, querytrainloader, querytestloader, (-1, 3, 32, 32), config["epochs"],
                                               optimizer=torch.optim.Adam(attacker.parameters(), lr=config["learning_rate"]),
                                               loss=torch.nn.CrossEntropyLoss())

                    # save & visualize model inference
                    title = f'A_{attacker_type}_{victim["model_name"]}_{victim["data"]}_{querytype}_{size}_k{k}'
                    percent = SaveVisualize(attacker, attacker_result, title)
                    results_dict={"Victim":victim["model_name"],
                                    "Dataset":victim["data"],
                                    "QueryType": querytype,
                                    "QuerySize": size,
                                    "Attacker": attacker_type,
                                    "Train Loss": percent[0],
                                    "Train Accuracy": percent[1],
                                    "Test Loss": percent[2],
                                    "Test Accuracy": percent[3]}

                    print(results_dict)
                    Results.append(results_dict)
                    print('-----------------------------------------------------------------------------')

In [ ]:
Results